In [1]:
import os

import pandas as pd
import numpy as np
import plotly.express as px

from scipy.stats import pearsonr

# Table of Contents

1. [Data loading](#data-loading)
2. [General analysis](#general-analysis)
    - [System sizes](#system-sizes)
    - [System investments](#system-investments)
3. [Comparison](#comparison)
    - [Total savings](#total-savings)
    - [Capacity costs savings](#capacity-costs-savings)
4. [Correlations](#correlations)
    - [On absolute savings](#on-absolute-savings)
    - [On relative savings](#on-relative-savings)

# Data loading

In [2]:
URI = os.getenv("DB_URI")

In [3]:
master = pd.read_sql("SELECT * FROM vea_industrial_load_profiles.master", URI)
master.set_index("id", inplace=True)
master.sort_index(inplace=True)

# in the authors database, the following columns are already present in master
# this will lead to problems later on, so we drop them here
# if you have created your own features by the "create_master_features.py", you
# do not need to worry about it
cols_to_drop = ["energy_costs_eur", "capacity_costs_eur", "total_costs_eur"]
for col_to_drop in cols_to_drop:
    try:
        master.drop(columns=col_to_drop, inplace=True)
    except KeyError as e:
        continue

master.head()

,sector_group_id,sector_group,capacity_price_over_2500h_eur_per_kw,energy_price_over_2500h_eur_per_kwh,capacity_price_under_2500h_eur_per_kw,energy_price_under_2500h_eur_per_kwh,zip_code,grid_level,number_of_peak_load,peak_load_kw,mean_load_kw,variance_kw,total_energy_kwh,full_load_hours_h,is_over_2500h,std_kw
id,,,,,,,,,,,,,,,,
0,B,Mining and quarrying,94.60,0.0060,14.67,0.0379,85,MS,1,2227.36,646.151302,423413.418622,5675793.04,2548.215394,True,650.702250
1,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,91,MS,1,1096.00,208.694900,109792.561757,1833176.00,1672.605839,False,331.349607
2,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,92,MS,1,816.00,129.444604,36662.220510,1137041.40,1393.433088,False,191.473812
3,B,Mining and quarrying,96.11,0.0052,9.92,0.0397,85,MS,1,701.60,114.201389,19954.089684,1003145.00,1429.796180,False,141.258946
4,C,Manufacturing industry,101.04,0.0078,12.60,0.0432,30,MS,2,3480.00,1845.483607,610080.465591,16210728.00,4658.255172,True,781.076479


In [4]:
profiles_over_2500h = master[master["is_over_2500h"]].index

In [5]:
baseline = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%base%%'", URI)
baseline["id"] = baseline["name"].str.split("_").str[0].astype(int)
baseline.set_index("id", inplace=True)
baseline = baseline[baseline.index.isin(profiles_over_2500h)]
baseline.sort_index(inplace=True)
baseline.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur
id,,,,,,,,,,,,,,,,,
0,0_baseline,1.021643e+06,271558.090967,41249.388603,2227.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.334450e+06,0.0,0.0
4,4_baseline,2.917931e+06,884063.074775,55353.715200,3480.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.857348e+06,0.0,0.0
5,5_baseline,2.797699e+05,59844.638186,8231.655936,568.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.478462e+05,0.0,0.0
6,6_baseline,2.243384e+06,567982.840899,35762.681088,2292.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.847129e+06,0.0,0.0
7,7_baseline,1.070598e+05,24777.926083,2599.534080,143.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.344373e+05,0.0,0.0


In [6]:
storage = pd.read_sql("SELECT * FROM vea_results.overview WHERE name LIKE '%%storage_only'", URI)
storage["id"] = storage["name"].str.split("_").str[0].astype(int)
storage.set_index("id", inplace=True)
storage = storage[storage.index.isin(profiles_over_2500h)]
storage["invest_per_tac"] = (storage["storage_invest_eur"] + storage["inverter_invest_eur"]) / baseline["total_yearly_costs_eur"]
storage.sort_index(inplace=True)
storage.head()

,name,energy_costs_eur,grid_energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,storage_invest_eur,storage_annuity_eur,storage_capacity_kwh,inverter_invest_eur,inverter_annuity_eur,inverter_capacity_kw,solar_invest_eur,solar_annuity_eur,solar_capacity_kwp,total_yearly_costs_eur,total_annuity_eur,total_invest_eur,invest_per_tac
id,,,,,,,,,,,,,,,,,,
0,0_storage_only,1.018851e+06,270816.128970,41249.388603,2227.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.330917e+06,0.0,0.0,0.0
4,4_storage_only,2.909959e+06,881647.601893,55353.715200,3480.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.846960e+06,0.0,0.0,0.0
5,5_storage_only,2.790055e+05,59681.128246,8231.655936,568.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.469182e+05,0.0,0.0,0.0
6,6_storage_only,2.237254e+06,566430.975213,35762.681088,2292.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.839448e+06,0.0,0.0,0.0
7,7_storage_only,1.067673e+05,24710.226832,2599.534080,143.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.340771e+05,0.0,0.0,0.0


# General analysis

In [7]:
total_profiles_analyzed = len(storage)
profiles_using_storage = storage[storage["inverter_invest_eur"] > 0]
n_profiles_using_storage = len(profiles_using_storage)

print(f"{total_profiles_analyzed=}")
print(f"{n_profiles_using_storage=}")
print(f"Percentage of profiles using storage: {((n_profiles_using_storage / total_profiles_analyzed) * 100):.2f} %")

total_profiles_analyzed=3644
n_profiles_using_storage=1145
Percentage of profiles using storage: 31.42 %


## System sizes

In [8]:
storage_size = profiles_using_storage["storage_capacity_kwh"].copy()
storage_size.name = "Storage capacity"
print(storage_size.describe().drop("count").to_markdown())

print("")
inverter_size = profiles_using_storage["inverter_capacity_kw"].copy()
inverter_size.name = "Inverter capacity"
print(inverter_size.describe().drop("count").to_markdown())


fig_df = profiles_using_storage.copy()
fig_df = fig_df.rename(columns={
    "storage_capacity_kwh": "Storage",
    "inverter_capacity_kw": "Inverter"})
fig = px.box(
    data_frame=fig_df,
    x=["Inverter", "Storage"],
    title="Battery system sizes")
fig.update_layout(xaxis_title="Capacity in kWh (storage) / kW (inverter)", yaxis_title="")
fig.update_xaxes(range=[0, 300])
fig.show()

|      |   Storage capacity |
|:-----|-------------------:|
| mean |        12.4516     |
| std  |        34.5134     |
| min  |         0.00157895 |
| 25%  |         1.50515    |
| 50%  |         4.26       |
| 75%  |        11.8102     |
| max  |       863.684      |

|      |   Inverter capacity |
|:-----|--------------------:|
| mean |            30.331   |
| std  |            59.061   |
| min  |             0.006   |
| 25%  |             4.94678 |
| 50%  |            13.4     |
| 75%  |            34.4     |
| max  |          1106       |


## System investments

In [9]:
storage_invest = profiles_using_storage["storage_invest_eur"].copy()
storage_invest.name = "Absolute storage investment"
print(storage_invest.describe().drop("count").to_markdown())

print("")
inverter_invest = profiles_using_storage["inverter_invest_eur"].copy()
inverter_invest.name = "Absolute inverter investment"
print(inverter_invest.describe().drop("count").to_markdown())


fig_df = profiles_using_storage.copy()
fig_df = fig_df.rename(columns={
    "storage_invest_eur": "Storage",
    "inverter_invest_eur": "Inverter"})
fig = px.box(
    data_frame=fig_df,
    x=["Inverter", "Storage"],
    title="Battery system investments")
fig.update_layout(xaxis_title="Storage system investments in €", yaxis_title="")
fig.update_xaxes(range=[0, 75000])
fig.show()

|      |   Absolute storage investment |
|:-----|------------------------------:|
| mean |                      3548.71  |
| std  |                      9836.32  |
| min  |                         0.45  |
| 25%  |                       428.967 |
| 50%  |                      1214.1   |
| 75%  |                      3365.91  |
| max  |                    246150     |

|      |   Absolute inverter investment |
|:-----|-------------------------------:|
| mean |                       5459.58  |
| std  |                      10631     |
| min  |                          1.08  |
| 25%  |                        890.421 |
| 50%  |                       2412     |
| 75%  |                       6192     |
| max  |                     199080     |


In [10]:
fig_df = storage.copy() * 100
fig = px.box(fig_df, "invest_per_tac", title="Investment per total yearly energy costs")
fig.update_xaxes(range=[0, 100], title="Ratio in %")

# Comparison

In [11]:
abs_diff = baseline.drop(columns="name") - storage.drop(columns="name")

# drop those that could not be optimized
abs_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

abs_diff.head()

,energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,grid_energy_costs_eur,inverter_annuity_eur,inverter_capacity_kw,inverter_invest_eur,invest_per_tac,solar_annuity_eur,solar_capacity_kwp,solar_invest_eur,storage_annuity_eur,storage_capacity_kwh,storage_invest_eur,total_annuity_eur,total_invest_eur,total_yearly_costs_eur
id,,,,,,,,,,,,,,,,,
0,2791.373626,0.0,0.0,741.961997,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3533.335623
4,7972.489180,0.0,0.0,2415.472882,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10387.962062
5,764.398525,0.0,0.0,163.509940,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,927.908465
6,6129.463279,0.0,0.0,1551.865686,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7681.328964
7,292.513185,0.0,0.0,67.699252,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.212437


In [12]:
# merge savings onto master (with features)
abs_diff_with_master = pd.merge(left=abs_diff, right=master, how="left", left_index=True, right_index=True)
abs_diff_with_master.head()

abs_diff_with_master["std_by_mean"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["mean_load_kw"]
abs_diff_with_master["std_by_peak"] = abs_diff_with_master["std_kw"] / abs_diff_with_master["peak_load_kw"]
abs_diff_with_master["peak_by_mean"] = abs_diff_with_master["peak_load_kw"] / abs_diff_with_master["mean_load_kw"]

In [13]:
rel_diff = (baseline.drop(columns="name") - storage.drop(columns="name")) / baseline.drop(columns="name")

# drop those that could not be optimized
rel_diff.dropna(subset="total_yearly_costs_eur", inplace=True)

rel_diff

,energy_costs_eur,grid_capacity_costs_eur,grid_capacity_kw,grid_energy_costs_eur,inverter_annuity_eur,inverter_capacity_kw,inverter_invest_eur,invest_per_tac,solar_annuity_eur,solar_capacity_kwp,solar_invest_eur,storage_annuity_eur,storage_capacity_kwh,storage_invest_eur,total_annuity_eur,total_invest_eur,total_yearly_costs_eur
id,,,,,,,,,,,,,,,,,
0,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002648
4,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002693
5,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002668
6,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002698
7,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5351,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002685
5352,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002702
5354,0.002732,0.00000,0.00000,0.002732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002704


In [14]:
# merge savings onto master (with features)
rel_diff_with_master = pd.merge(left=rel_diff, right=master, how="left", left_index=True, right_index=True)
rel_diff_with_master.head()

rel_diff_with_master["std_by_mean"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["mean_load_kw"]
rel_diff_with_master["std_by_peak"] = rel_diff_with_master["std_kw"] / rel_diff_with_master["peak_load_kw"]
rel_diff_with_master["peak_by_mean"] = rel_diff_with_master["peak_load_kw"] / rel_diff_with_master["mean_load_kw"]

## Total savings

In [15]:
abs_yearly_savings = abs_diff["total_yearly_costs_eur"].copy()
abs_yearly_savings.name = "Total yearly savings in eur"
print(abs_yearly_savings.describe().drop("count").to_markdown())

fig_df = abs_diff.copy()
fig_df = fig_df.rename(columns={"total_yearly_costs_eur": "Savings"})
fig = px.box(
    data_frame=fig_df,
    x=["Savings"],
    title="Total yearly savings")
fig.update_layout(xaxis_title="Total yearly savings in €", yaxis_title="")
fig.update_xaxes(range=[0, 15e3])
fig.show()

|      |   Total yearly savings in eur |
|:-----|------------------------------:|
| mean |                    2699.66    |
| std  |                    5893.71    |
| min  |                       3.70576 |
| 25%  |                     362.838   |
| 50%  |                    1009.17    |
| 75%  |                    2706.51    |
| max  |                  120014       |


In [16]:
rel_yearly_savings = 100 * rel_diff["total_yearly_costs_eur"].copy()
rel_yearly_savings.name = "Relative yearly savings in %"
print(rel_yearly_savings.describe().drop("count").to_markdown())
print("-----------------------")
n_profiles_1pct_yearly_savings = len(rel_diff[rel_diff["total_yearly_costs_eur"] > 0.01])
print(f"Number of profiles with more than 1% yearly savings: {n_profiles_1pct_yearly_savings}")
perc_profiles_1pct_yearly_savings = (n_profiles_1pct_yearly_savings / len(rel_diff)) * 100
print(f"Percentage of profiles with more than 1% yearly savings: {perc_profiles_1pct_yearly_savings:.2f} %")
n_profiles_2pct_yearly_savings = len(rel_diff[rel_diff["total_yearly_costs_eur"] > 0.02])
print(f"Number of profiles with more than 2% yearly savings: {n_profiles_2pct_yearly_savings}")
perc_profiles_2pct_yearly_savings = (n_profiles_2pct_yearly_savings / len(rel_diff)) * 100
print(f"Percentage of profiles with more than 2% yearly savings: {perc_profiles_2pct_yearly_savings:.2f} %")
n_profiles_3pct_yearly_savings = len(rel_diff[rel_diff["total_yearly_costs_eur"] > 0.03])
print(f"Number of profiles with more than 3% yearly savings: {n_profiles_3pct_yearly_savings}")
perc_profiles_3pct_yearly_savings = (n_profiles_3pct_yearly_savings / len(rel_diff)) * 100
print(f"Percentage of profiles with more than 3% yearly savings: {perc_profiles_3pct_yearly_savings:.2f} %")

fig_df = pd.DataFrame()
fig_df = rel_diff.copy() * 100
fig_df = fig_df.rename(columns={"total_yearly_costs_eur": "Savings"})
fig = px.box(
    data_frame=fig_df,
    x="Savings",
    title="Relative yearly savings")
fig.update_layout(xaxis_title="Relative yearly savings in %", yaxis_title="")
fig.update_xaxes(range=[0, 15])
fig.show()

|      |   Relative yearly savings in % |
|:-----|-------------------------------:|
| mean |                      0.291266  |
| std  |                      0.0932143 |
| min  |                      0.257192  |
| 25%  |                      0.26845   |
| 50%  |                      0.269868  |
| 75%  |                      0.271973  |
| max  |                      2.10708   |
-----------------------
Number of profiles with more than 1% yearly savings: 12
Percentage of profiles with more than 1% yearly savings: 0.33 %
Number of profiles with more than 2% yearly savings: 1
Percentage of profiles with more than 2% yearly savings: 0.03 %
Number of profiles with more than 3% yearly savings: 0
Percentage of profiles with more than 3% yearly savings: 0.00 %


## Capacity costs savings

In [17]:
abs_cap_costs_yearly_savings = abs_diff["grid_capacity_costs_eur"].copy()
abs_cap_costs_yearly_savings.name = "Total yearly grid capacity costs savings in %"
print(abs_cap_costs_yearly_savings.describe().drop("count").to_markdown())

total_cap_saving_fig_df = abs_diff.copy()
total_cap_saving_fig_df = total_cap_saving_fig_df.rename(columns={"grid_capacity_costs_eur": "Savings"})
total_cap_savings_fig = px.box(
    data_frame=total_cap_saving_fig_df,
    x="Savings",
    title="Yearly capacity costs savings")
total_cap_savings_fig.update_layout(xaxis_title="Savings in €", yaxis_title="")
total_cap_savings_fig.update_xaxes(range=[0, 10e3])
total_cap_savings_fig.show()

|      |   Total yearly grid capacity costs savings in % |
|:-----|------------------------------------------------:|
| mean |                                         313.535 |
| std  |                                        1387.53  |
| min  |                                           0     |
| 25%  |                                           0     |
| 50%  |                                           0     |
| 75%  |                                         104.163 |
| max  |                                       52009     |


In [18]:
rel_cap_costs_yearly_savings = 100 * rel_diff["grid_capacity_costs_eur"].copy()
rel_cap_costs_yearly_savings.name = "Relative yearly grid capacity costs savings in %"
print(rel_cap_costs_yearly_savings.describe().drop("count").to_markdown())

rel_cap_savings_fig_df = pd.DataFrame()
rel_cap_savings_fig_df = rel_diff.copy() * 100
rel_cap_savings_fig_df.rename(columns={"grid_capacity_costs_eur": "Savings"}, inplace=True)
rel_cap_savings_fig = px.box(
    data_frame=rel_cap_savings_fig_df,
    x="Savings",
    title="Relative yearly capacity cost savings")
rel_cap_savings_fig.update_layout(xaxis_title="Savings in %", yaxis_title="")
rel_cap_savings_fig.update_xaxes(range=[0, 75])
rel_cap_savings_fig.show()

|      |   Relative yearly grid capacity costs savings in % |
|:-----|---------------------------------------------------:|
| mean |                                            1.85823 |
| std  |                                            4.45138 |
| min  |                                            0       |
| 25%  |                                            0       |
| 50%  |                                            0       |
| 75%  |                                            1.42885 |
| max  |                                           44.7172  |


# Correlations

## On absolute savings

In [19]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
abs_correlations_df = abs_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(abs_correlations_df, title="Correlation coefficients for total yearly savings")

In [20]:
fig_df = abs_correlations_df[["total_yearly_costs_eur"]].round(2)
fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
fig = px.bar(
    data_frame=fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and total yearly savings")
fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")

## On relative savings

In [21]:
cols_to_drop = [
    "grid_level",
    "zip_code",
    "sector_group_id",
    "sector_group",
    "solar_invest_eur",
    "solar_annuity_eur",
    "solar_capacity_kwp"]
rel_correlations_df = rel_diff_with_master.drop(columns=cols_to_drop).corr()
px.imshow(rel_correlations_df, title="Correlation coefficients for relative yearly savings")

In [22]:
fig_df = rel_correlations_df[["total_yearly_costs_eur"]].round(2)
fig_df.sort_values("total_yearly_costs_eur", inplace=True, ascending=False)
fig_df.dropna(inplace=True)
fig = px.bar(
    data_frame=fig_df,
    y="total_yearly_costs_eur",
    text_auto=True,
    title="Correlation between different load profile characteristics and relative yearly savings")
fig.update_layout(yaxis_title="Correlation coefficient", xaxis_title="Variable")

In [23]:
df = pd.DataFrame()
i = 0
for var in rel_correlations_df.index:
    if np.inf in rel_diff_with_master[var]:
        continue
    elif -np.inf in rel_diff_with_master[var]:
        continue
    elif np.nan in rel_diff_with_master[var]:
        continue
    elif rel_diff_with_master[var].isin([np.nan, np.inf, -np.inf]).any():
        continue
    if "storage" in var or "inverter" in var:
        continue

    corr, p_value = pearsonr(y=rel_diff_with_master["total_yearly_costs_eur"], x=rel_diff_with_master[var])
    df.loc[i, "var"] = var
    df.loc[i, "corr"] = corr
    df.loc[i, "p_value"] = p_value
    i += 1

df.sort_values("corr", ascending=False, ignore_index=True)

/tmp/ipykernel_56261/2488967782.py:15: ConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,var,corr,p_value
0,total_yearly_costs_eur,1.000000,0.000000e+00
1,grid_capacity_costs_eur,0.772357,0.000000e+00
2,grid_capacity_kw,0.772357,0.000000e+00
3,capacity_price_under_2500h_eur_per_kw,0.602824,0.000000e+00
4,energy_price_over_2500h_eur_per_kwh,0.289900,1.711133e-71
5,capacity_price_over_2500h_eur_per_kw,0.230994,2.453462e-45
6,energy_price_under_2500h_eur_per_kwh,0.203932,1.642028e-35
7,peak_by_mean,0.120949,2.379397e-13
8,std_by_mean,-0.007822,6.368994e-01
9,variance_kw,-0.022499,1.745047e-01


## Scatterplots

In [24]:
cap_vs_savings_fig_df = rel_diff_with_master.copy()
cap_vs_savings_fig_df.loc[cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_eur"] = cap_vs_savings_fig_df.loc[cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_over_2500h_eur_per_kw"].copy()
cap_vs_savings_fig_df.loc[~cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_eur"] = cap_vs_savings_fig_df.loc[~cap_vs_savings_fig_df["is_over_2500h"], "capacity_price_under_2500h_eur_per_kw"].copy()
cap_vs_savings_fig_df["total_yearly_costs_eur"] *= 100
cap_vs_savings_fig_df = cap_vs_savings_fig_df[cap_vs_savings_fig_df.index.isin(profiles_using_storage.index)]
cap_vs_savings_fig_df = cap_vs_savings_fig_df[cap_vs_savings_fig_df["std_by_mean"] < 2]
# cap_vs_savings_fig_df = cap_vs_savings_fig_df[cap_vs_savings_fig_df["capacity_price_eur"] < 200]

cap_vs_savings_fig = px.scatter(
    data_frame=cap_vs_savings_fig_df,
    x="std_by_mean",
    y="total_yearly_costs_eur",
    title="Grid capacity prices vs relative yearly savings",
    color="is_over_2500h",
    trendline="ols",
    trendline_color_override="black")
cap_vs_savings_fig.update_xaxes(title="Grid capacity costs in eur / kW")
cap_vs_savings_fig.update_yaxes(title="Relative yearly savings in %")
cap_vs_savings_fig.update_layout(showlegend=False)
cap_vs_savings_fig.show()